In [224]:
import pandas as pd
import numpy as np
import plotly.express as px

#Importando todos los datos
df_afp=pd.read_csv("./data/armed-forces-personnel.csv")
df_afp1=df_afp.style.set_caption("Personal de las fuerzas armadas")
df_met=pd.read_csv("./data/military-expenditure-total.csv")
df_mepc=pd.read_csv("./data/military-spending-per-capita.csv")
df_ngpr=pd.read_csv("./data/natural-gas-proved-reserves.csv")
df_opr=pd.read_csv("./data/oil-proved-reserves.csv")

#Haciendo merge de todos los datos para no perder ni un solo dato de todas los datasets
df_all=pd.merge(df_afp,df_met,on=["Entity","Year","Code"],how="left")
df_all=pd.merge(df_all,df_mepc,on=["Entity","Year","Code"],how="left")
df_all=pd.merge(df_all,df_ngpr,on=["Entity","Year","Code"],how="left")
df_all=pd.merge(df_all,df_opr,on=["Entity","Year","Code"],how="left")
#df_all.to_csv("El mero mero caguamero.csv") #esta línea se descomenta si se quiere ver cómo queda el CSV

Entendiendo la estructura más básica de cada columna por cada Dataset `(se pretende definir dinámica, comportamiento y estructura de cada dataset)`

# 1.0 Para personal de las fuerzas armadas
`Dataset: "armed-forces-personnel.csv"`

In [225]:
print("\n==== [head] de: "+df_afp1.caption+" ====")
print(df_afp1.data.head())
print("\n==== [dtypes] de: "+df_afp1.caption+" ====")
print(df_afp1.data.dtypes)
print("\n==== [groupby-count de Year] de: "+df_afp1.caption+" ====")
print(df_afp1.data.groupby(["Entity","Code"])["Year"].count().sort_values(ascending=False).head())
print(df_afp1.data.groupby(["Entity","Code"])["Year"].count().sort_values(ascending=False).tail())


==== [head] de: Personal de las fuerzas armadas ====
        Entity Code  Year  Armed forces personnel, total
0  Afghanistan  AFG  1985                          47000
1  Afghanistan  AFG  1989                          55000
2  Afghanistan  AFG  1990                          58000
3  Afghanistan  AFG  1991                          45000
4  Afghanistan  AFG  1992                          45000

==== [dtypes] de: Personal de las fuerzas armadas ====
Entity                           object
Code                             object
Year                              int64
Armed forces personnel, total     int64
dtype: object

==== [groupby-count de Year] de: Personal de las fuerzas armadas ====
Entity       Code
Zimbabwe     ZWE     32
Philippines  PHL     32
Japan        JPN     32
Jamaica      JAM     32
Italy        ITA     32
Name: Year, dtype: int64
Entity       Code
Eswatini     SWZ     11
Bhutan       BTN     10
South Sudan  SSD     10
Palestine    PSE      9
Maldives     MDV      3
Na

## 1.1 Definiendo MTC y MD

1. Medidas de Tendencia Central: 
   *  Definiendo el promedio
   *  Contando datos por país agrupados
   *  Periodos por cuartil
   *  Valores mínimo y máximo por país
   

2. Medidas de Dispersión: Agregando desviación estándar

In [226]:
print("\n==== [describe] de: "+df_afp1.caption+" ====") 
print(df_afp.groupby(["Entity","Code"])["Armed forces personnel, total"].describe())


==== [describe] de: Personal de las fuerzas armadas ====
                              count          mean           std         min  \
Entity              Code                                                      
Afghanistan         AFG        31.0  2.154127e+05  1.501796e+05     27000.0   
Albania             ALB        29.0  3.284103e+04  2.608547e+04      8000.0   
Algeria             DZA        32.0  2.649062e+05  8.167883e+04    126000.0   
Angola              AGO        32.0  1.180594e+05  1.556433e+04     49500.0   
Antigua and Barbuda ATG        25.0  1.560000e+02  6.271629e+01         0.0   
...                             ...           ...           ...         ...   
Vietnam             VNM        31.0  6.264839e+05  2.061328e+05    495000.0   
World               OWID_WRL   32.0  2.719570e+07  2.204732e+06  22209230.0   
Yemen               YEM        32.0  1.015156e+05  4.584353e+04     20000.0   
Zambia              ZMB        32.0  1.802500e+04  2.984260e+03     16000

## 1.2 Buscando extraer el TOP(N) de Países con mayor cantidad de personal militar por año

In [227]:
#Aquí nos damos cuenta que existen valores que no queremos como los datos del Mundo
prueba=df_afp.groupby(["Year","Entity"])["Armed forces personnel, total"].max()
print(prueba.sort_index(ascending=False).sort_values(ascending=False))
prueba=prueba.reset_index().sort_values(by=["Year","Armed forces personnel, total"])
prueba


Year  Entity             
1996  World                  30196640
1995  World                  30182200
1998  World                  29926660
1997  World                  29863220
1999  World                  29584140
                               ...   
2019  Congo                         0
2014  Libya                         0
2017  Iceland                       0
2019  Seychelles                    0
      Antigua and Barbuda           0
Name: Armed forces personnel, total, Length: 5659, dtype: int64


,Year,Entity,"Armed forces personnel, total"
4,1985,Antigua and Barbuda,100
51,1985,Gambia,480
8,1985,Bahamas,500
13,1985,Belize,600
84,1985,Luxembourg,700
...,...,...,...
5650,2019,Upper middle income,8574000
5572,2019,Lower middle income,10235000
5583,2019,Middle income,18809000
5570,2019,Low and middle income,21987000


En esta parte necesitaba filtrar por año los países con mayor cantidad de personal evitando datos del mundo y por región; por lo que para este punto utilicé el 2019 como referencia para luego aplicarlo en una función y traer por año los países con mayor cantidad de contribución de personal

In [228]:
years=prueba["Year"].unique()
print(years.max())
list_to_avoid=["World","Low and middle income","Middle income","Lower middle income","Upper middle income",
            "East Asia and Pacific","High income","Europe and Central Asia","South Asia","Middle East and North Africa",
            "Low income","Latin America and Caribbean","Sub-Saharan Africa","European Union","North America"]
prueba2=prueba.where(prueba["Year"]==years.max()).dropna().sort_values(by="Armed forces personnel, total",ascending=False)
#sacando el top 15 de países con mayor personal para el 2019, el False es para evitar la lista de regiones y el mundo
prueba2[prueba2.Entity.str.contains('|'.join(list_to_avoid))==False].head(15) 


2019


,Year,Entity,"Armed forces personnel, total"
5549,2019.0,India,3045000.0
5508,2019.0,China,2535000.0
5598,2019.0,North Korea,1469000.0
5613,2019.0,Russia,1454000.0
5649,2019.0,United States,1388000.0
5602,2019.0,Pakistan,943000.0
5523,2019.0,Egypt,836000.0
5496,2019.0,Brazil,762000.0
5550,2019.0,Indonesia,676000.0
5551,2019.0,Iran,650000.0


Usando todo lo anterior se llegó al siguiente código [MODO LOCOHON ACTIVADO]

In [233]:
#Quiero filtrar esta lista de mi dataset
list_to_avoid=["World","Low and middle income","Middle income","Lower middle income","Upper middle income",
            "East Asia and Pacific","High income","Europe and Central Asia","South Asia","Middle East and North Africa",
            "Low income","Latin America and Caribbean","Sub-Saharan Africa","European Union","North America"]
#En este paso lo filtramos y evitamos la lista de arriba
df_afp_locochon=df_afp[df_afp.Entity.str.contains('|'.join(list_to_avoid))==False]
#Aquí creamos un DataFrame vacío para hacerle merge con las mismas columnas
df_afp_topN=pd.DataFrame(columns=df_afp_locochon.columns)
#Decidimos traer 15 países por año
First_N_Values=15
for year in years:
    #Recorremos por año el dataset locochon y quitamos los NaN values, ordenamos de mayor a menor y retiramos los primers N valores
    data_per_year=df_afp_locochon.where(df_afp_locochon["Year"]==year).dropna().sort_values(by="Armed forces personnel, total",ascending=False).head(First_N_Values)
    #Procedemos a hacer el merge tipo outer para no perder ningun dato
    df_afp_topN=df_afp_topN.merge(data_per_year,how="outer")

#Comprobamos
df_afp_topN.iloc[0:30]

,Entity,Code,Year,"Armed forces personnel, total"
0,China,CHN,1985.0,3900000.0
1,United States,USA,1985.0,2151600.0
2,India,IND,1985.0,1260000.0
3,Vietnam,VNM,1985.0,1027000.0
4,Iraq,IRQ,1985.0,1000000.0
5,North Korea,PRK,1985.0,838000.0
6,Turkey,TUR,1985.0,630000.0
7,Iran,IRN,1985.0,610000.0
8,South Korea,KOR,1985.0,598000.0
9,Pakistan,PAK,1985.0,482800.0


## 1.3 Una vez terminado el TOP 15 de Países por año comenzamos a analizar su crecimiento en poder militar

### 1.3.1 Gráfico de Barras

In [260]:
fig_bar_afp=px.bar(df_afp_topN,y="Year",x="Armed forces personnel, total",color="Entity",
                    height=1000,text="Entity",text_auto='.5s',pattern_shape="Entity",
                    orientation='h',title="Fuerzas armadas por año por país")
fig_bar_afp.show()

### 1.3.2 Histograma

Histograma de años reportados por cada país

In [278]:
fig_hist_afp=px.histogram(df_afp_topN.groupby(["Code","Entity"])["Year"].count().sort_values(ascending=False).reset_index(),
                        x="Code",y="Year",color="Entity",text_auto='1s',pattern_shape="Entity",
                        title="¿Cuántos años se reportan en este Dataset por país?")
fig_hist_afp.show()

In [317]:
data=df_afp.groupby(["Code","Entity","Year"])["Armed forces personnel, total"].max().reset_index()
data_Mexico=data.where(data["Entity"]=="Mexico").dropna()
data_Mexico=data_Mexico.merge(data.where(data["Entity"]=="United States").dropna(),how="outer")
fig_hist_data_Mexico=px.histogram(data_Mexico,x="Year",y="Armed forces personnel, total",color="Entity",
            text_auto='1s',pattern_shape="Entity",title="Histograma semilogarítmico de fuerza militar de México vs USA",
            nbins=20,log_y=True)
fig_hist_data_Mexico.show()

In [318]:
data_Mexico

,Code,Entity,Year,"Armed forces personnel, total"
0,MEX,Mexico,1985.0,129100.0
1,MEX,Mexico,1989.0,154000.0
2,MEX,Mexico,1990.0,175000.0
3,MEX,Mexico,1991.0,175000.0
4,MEX,Mexico,1992.0,175000.0
...,...,...,...,...
59,USA,United States,2015.0,1347300.0
60,USA,United States,2016.0,1348400.0
61,USA,United States,2017.0,1359000.0
62,USA,United States,2018.0,1379800.0


Histograma de comparativa a Nivel Mundial

In [231]:
fig_df_afp=px.histogram(df_afp.groupby(["Code","Entity"])["Armed forces personnel, total"].max().sort_values(ascending=False).head(21).reset_index(),x="Code",y="Armed forces personnel, total",color="Entity",
                        title='Histograma de Top 20 de Fuerzas Armadas por país vs comparativa a nivel Mundial',opacity=0.8,log_y=True,pattern_shape="Code",marginal="rug",text_auto=True)
fig_df_afp.update_layout(bargap=0.2)
fig_df_afp.show()

Creando gráficos de Histogramas y Boxplots

In [325]:
fig_boxplot_afp=px.box(data_Mexico,x="Code",y="Armed forces personnel, total",points="all",
                        notched=True,log_y=True)
fig_boxplot_afp.show()

Aquí tengo una duda: ¿Por qué en este gráfico se ve así para México?

# Analizando el Merge de todo el dataset

Favor de abrir todo el editor de texto para ver los resultados

In [326]:
print("\n==== [head] del dataset con merge ====")
print(df_all.head())
print("\n==== [dtypes] del dataset con merge ====")
print(df_all.dtypes)
print("\n==== [groupby-count de Year] del dataset con merge ====")
print("\n\t\t\t\t==== HEAD ====")
print(df_all.groupby(["Entity","Code"])["Year"].count().sort_values(ascending=False).head())
print("\n\t\t\t\t==== TAIL ====")
print(df_all.groupby(["Entity","Code"])["Year"].count().sort_values(ascending=False).tail())


==== [head] del dataset con merge ====
        Entity Code  Year  Armed forces personnel, total  \
0  Afghanistan  AFG  1985                          47000   
1  Afghanistan  AFG  1989                          55000   
2  Afghanistan  AFG  1990                          58000   
3  Afghanistan  AFG  1991                          45000   
4  Afghanistan  AFG  1992                          45000   

   military_expenditure  military_expenditure_per_capita  \
0                   NaN                              NaN   
1                   NaN                              NaN   
2                   NaN                              NaN   
3                   NaN                              NaN   
4                   NaN                              NaN   

   Gas - Proved reserves  Oil - Proved reserves  
0                    NaN                    NaN  
1                    NaN                    NaN  
2                    NaN                    NaN  
3                    NaN              